In [ ]:
from s123456 import Problem
import networkx as nx
import matplotlib.pyplot as plt
from src.instance import Solution

In [ ]:
problem = Problem(10, density=0.2)

In [ ]:
G = problem.graph
pos_dict = {k: v['pos'] for k, v in G.nodes(data=True)}
gold_dict = {n: data['gold'] for n, data in G.nodes(data=True)}
paths_dict = nx.shortest_path(problem.graph, source=0, weight='dist')

In [ ]:
paths_dict

In [ ]:
fig, axs = plt.subplots(len(paths_dict) // 4 + 1, 4, figsize=(15, 10))
# draw each shortest path in a different colour
paths = [p for p in paths_dict.values() if len(p) > 1]
n = len(paths)
cmap = plt.cm.get_cmap('tab10', n)
for i, (path, ax) in enumerate(zip(paths, axs.flatten())):
    color = cmap(i / (n - 1)) if n > 1 else cmap(0)
    edgelist = [(u, v) for u, v in zip(path, path[1:])]
    nx.draw(G, with_labels=True, pos=pos_dict, ax=ax)
    nx.draw_networkx_edges(G, pos=pos_dict, edgelist=edgelist,
                           edge_color=[color] * len(edgelist),
                           width=2.5, alpha=0.8, ax=ax)

In [ ]:
solution_1 = Solution(G=G)
str(solution_1)

In [ ]:
solution_2 = Solution(paths_dict=paths_dict, gold_dict=gold_dict)
str(solution_2)